In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/semantic_parsing/tree_slu/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install tensorflow-hub

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import pprint
import logging
import time
import nltk
import os
import random

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.14.0
GPU Enabled: True


In [0]:
# stream data from text files
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      text_raw, text_tokenized, label = line.split('\t')
      text_tokenized = text_tokenized.lower().split()
      label = label.replace('[', '[ ').lower().split()
      #source = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in text_tokenized]
      target = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in label]
      target_in = [1] + target
      target_out = target + [2]
      yield (text_tokenized, (target_in, target_out))

In [0]:
def dataset(is_training, params):
  _shapes = ([None], ([None], [None]))
  _types = (tf.string, (tf.int32, tf.int32))
  _pads = ('<pad>', (0, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['train_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['eval_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def clip_grads(loss):
    variables = tf.trainable_variables()
    pprint.pprint(variables)
    grads = tf.gradients(loss, variables)
    clipped_grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    return zip(clipped_grads, variables)


def rnn_cell():
    def cell_fn():
        cell = tf.nn.rnn_cell.GRUCell(params['rnn_units'],
                                      kernel_initializer=tf.orthogonal_initializer())
        return cell
    if params['num_layers'] > 1:
      return tf.nn.rnn_cell.MultiRNNCell([cell_fn() for _ in range(params['num_layers'])])
    else:
      return cell_fn()

  
def dec_cell(enc_out, enc_seq_len):
    attn1 = tf.contrib.seq2seq.BahdanauAttention(
        num_units = params['rnn_units'],
        memory = enc_out,
        memory_sequence_length = enc_seq_len,
        normalize=False)
    
    attn2 = tf.contrib.seq2seq.LuongAttention(
        num_units = params['rnn_units'],
        memory = enc_out,
        memory_sequence_length = enc_seq_len,
        scale=False)
    
    return tf.contrib.seq2seq.AttentionWrapper(
        cell = rnn_cell(),
        attention_mechanism = (attn1, attn2),
        attention_layer_size = (params['rnn_units'], params['rnn_units']))

In [0]:
class TiedDense(tf.layers.Layer):
  def __init__(self, tied_embed, out_dim):
    super().__init__()
    self.tied_embed = tied_embed
    self.out_dim = out_dim
  
  def build(self, input_shape):
    self.bias = self.add_weight(name='bias',
                                shape=[self.out_dim],
                                trainable=True)
    self.proj_W = self.add_weight(name='proj_W',
                                  shape=[2*params['rnn_units'], params['rnn_units']],
                                  trainable=True)
    self.proj_b = self.add_weight(name='proj_b',
                                  shape=[params['rnn_units']],
                                  trainable=True)
    super().build(input_shape)
  
  def call(self, inputs):
    x = tf.nn.elu(tf.nn.bias_add(tf.matmul(inputs, self.proj_W), self.proj_b))
    x = tf.matmul(x, self.tied_embed, transpose_b=True)
    x = tf.nn.bias_add(x, self.bias)
    return x
  
  def compute_output_shape(self, input_shape):
    return input_shape[:-1].concatenate(self.out_dim)

In [0]:
def forward(raw_texts, labels, mode):
    vocab = tf.contrib.lookup.index_table_from_file(
        params['vocab_tgt_path'], num_oov_buckets=1)
    words = vocab.lookup(raw_texts)
    words_len = tf.count_nonzero(words, 1, dtype=tf.int32)
    
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    batch_sz = tf.shape(words)[0]
    
    
    with tf.variable_scope('Embedding'):
        elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
        e = elmo(inputs={'tokens':raw_texts, 'sequence_len':words_len,}, signature="tokens", as_dict=True)['lstm_outputs1']
        e = tf.layers.dropout(e, params['dropout_rate'], training=is_training)
        e = tf.layers.dense(e, params['rnn_units'], tf.nn.elu)
        
        embedding = tf.Variable(np.load('../vocab/word.npy'),
                                dtype=tf.float32,
                                name='glove')
        x = tf.nn.embedding_lookup(embedding, words)
        x = tf.concat((x, e), -1)
        x = tf.layers.dropout(x, params['dropout_rate'], training=is_training)
    
    
    with tf.variable_scope('Encoder'):
        encoder = tf.keras.layers.CuDNNGRU(params['rnn_units'],
                                           return_state=True,
                                           return_sequences=True)
        encoder = tf.keras.layers.Bidirectional(encoder)
        enc_out, state_fw, state_bw = encoder(x)
        enc_state = (state_fw + state_bw) / 2
        if params['num_layers'] > 1:
          enc_state = tuple(params['num_layers'] * [enc_state])
    
    
    with tf.variable_scope('Decoder'):
        output_proj = TiedDense(embedding, len(params['tgt2idx'])+1)
        
        if is_training or (mode == tf.estimator.ModeKeys.EVAL):
            dec_inputs, dec_outputs = labels
            dec_seq_len = tf.count_nonzero(dec_inputs, 1, dtype=tf.int32)
            dec_inputs = tf.nn.embedding_lookup(embedding, dec_inputs)
            dec_inputs = tf.layers.dropout(dec_inputs, params['dropout_rate'], training=is_training)
            cell = dec_cell(enc_out, words_len)
            
            init_state = cell.zero_state(batch_sz, tf.float32).clone(
                cell_state=enc_state)
            
            helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = dec_inputs,
                sequence_length = dec_seq_len)
            decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = cell,
                helper = helper,
                initial_state = init_state,
                output_layer = output_proj)
            decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = decoder,
                maximum_iterations = tf.reduce_max(dec_seq_len))
            
            return decoder_output.rnn_output
        else:
            enc_out_t = tf.contrib.seq2seq.tile_batch(enc_out, params['beam_width'])
            enc_state_t = tf.contrib.seq2seq.tile_batch(enc_state, params['beam_width'])
            enc_seq_len_t = tf.contrib.seq2seq.tile_batch(words_len, params['beam_width'])
            
            cell = dec_cell(enc_out_t, enc_seq_len_t)
            
            init_state = cell.zero_state(batch_sz*params['beam_width'], tf.float32).clone(
                cell_state=enc_state_t)
            
            decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = cell,
                embedding = embedding,
                start_tokens = tf.tile(tf.constant([1], tf.int32), [batch_sz]),
                end_token = 2,
                initial_state = init_state,
                beam_width = params['beam_width'],
                output_layer = output_proj)
            decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = decoder,
                maximum_iterations = 80,)
            
            return decoder_output.predicted_ids[:, :, 0]

In [0]:
def model_fn(features, labels, mode, params):
    logits_or_ids = forward(features, labels, mode)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=logits_or_ids)
    
    dec_inputs, dec_outputs = labels
    loss_op = tf.contrib.seq2seq.sequence_loss(logits = logits_or_ids,
                                               targets = dec_outputs,
                                               weights = tf.to_float(tf.sign(dec_outputs)))  
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step=tf.train.get_or_create_global_step()
        
        decay_lr = tf.train.exponential_decay(
            params['lr'], global_step, 1000, .96)
        
        train_op = tf.train.AdamOptimizer(decay_lr).apply_gradients(
            clip_grads(loss_op), global_step=global_step)
        
        hook = tf.train.LoggingTensorHook({'lr': decay_lr}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[hook],)

In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
params = {
    'model_dir': '../model/gru_seq2seq',
    'log_path': '../log/gru_seq2seq.txt',
    'train_path': '../data/train.tsv',
    'test_path': '../data/test.tsv',
    'vocab_src_path': '../vocab/source.txt',
    'vocab_tgt_path': '../vocab/target.txt',
    'model_path': '../model/',
    'dropout_rate': 0.2,
    'rnn_units': 300,
    'num_layers': 2,
    'beam_width': 10,
    'lr': 4e-4,
    'clip_norm': .1,
    'buffer_size': 31279,
    'train_batch_size': 32,
    'eval_batch_size': 128,
    'num_patience': 5,
}

In [0]:
params['tgt2idx'] = get_vocab(params['vocab_tgt_path'])
params['idx2tgt'] = {idx: tgt for tgt, idx in params['tgt2idx'].items()}

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [0]:
def minimal_test(estimator):
  test_str = [['what', 'times', 'are', 'the', 'nutcracker', 'show', 'playing', 'near', 'me']]
  predicted = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    x = np.array(test_str), shuffle = False)))
  print('-'*12)
  print('minimal test')
  print('utterance:', ' '.join(test_str[0]))
  predicted = ' '.join([params['idx2tgt'].get(idx, len(params['idx2tgt'])) for idx in predicted[0]])
  predicted = predicted.replace('<end>', '').strip()
  print('parsed:', predicted)
  print()
  try:
    nltk.tree.Tree.fromstring(predicted.replace('[ ', '(').replace(' ]', ')')).pretty_print()
  except:
    pass
  print('-'*12)

In [15]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
eval_steps = params['buffer_size']//params['train_batch_size'] + 1
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=tf.estimator.RunConfig(save_checkpoints_steps=eval_steps),
  params=params)

best_acc = .0
history_acc = []
tf.enable_eager_execution()

while True:
  estimator.train(input_fn=lambda: dataset(is_training=True, params=params))

  minimal_test(estimator)

  # Evaluation
  labels = [label for _, (_, label) in dataset(is_training=False, params=params)]
  seq_lens = [tf.argmax(tf.cast(tf.equal(l, 2), tf.int32), axis=1) for l in labels]
  labels = [j for i in labels for j in i.numpy()]
  seq_lens = [j for i in seq_lens for j in i.numpy()]

  preds = list(estimator.predict(input_fn=lambda: dataset(is_training=False, params=params)))

  assert len(preds) == len(labels)

  res = [np.all(p[:seq_len+1] == l[:seq_len+1]) for p, l, seq_len in zip(preds, labels, seq_lens)]
  acc = np.asarray(res).mean()
  logger.info("Evaluation: Testing (Exact Match) Accuracy: {:.3f}".format(acc))
  history_acc.append(acc)

  if acc > best_acc:
    best_acc = acc
  logger.info("Best (Exact Match) Accuracy: {:.3f}".format(best_acc))

  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing (Exact Match) Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

I0815 01:02:57.840075 140072451344256 estimator.py:209] Using config: {'_model_dir': '../model/gru_seq2seq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 978, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f64d0ca2cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
W0815 01:02:57.883921 140072451344256 deprecation.py:323] From /usr/local/lib/python3.6/di

[<tf.Variable 'Embedding/dense/kernel:0' shape=(1024, 300) dtype=float32_ref>,
 <tf.Variable 'Embedding/dense/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Embedding/glove:0' shape=(8692, 300) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional/forward_cu_dnngru/kernel:0' shape=(600, 900) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/forward_cu_dnngru/recurrent_kernel:0' shape=(300, 900) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/forward_cu_dnngru/bias:0' shape=(1800,) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/backward_cu_dnngru/kernel:0' shape=(600, 900) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/backward_cu_dnngru/recurrent_kernel:0' shape=(300, 900) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/backward_cu_dnngru/bias:0' shape=(1800,) dtype=float32>,
 <tf.Variable 'Decoder/memory_layer/kernel:0' shape=(600, 300) dtype=float32_ref>,
 <tf.Variable 'Decoder/memory_layer_1/kernel:0' shape=(600, 300) dtype=float32_ref>,
 <tf.Var

I0815 01:03:05.207180 140072451344256 estimator.py:1147] Done calling model_fn.
I0815 01:03:05.211488 140072451344256 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0815 01:03:06.569250 140072451344256 monitored_session.py:240] Graph was finalized.
I0815 01:03:09.288105 140072451344256 session_manager.py:500] Running local_init_op.
I0815 01:03:09.376079 140072451344256 session_manager.py:502] Done running local_init_op.
I0815 01:03:15.235157 140072451344256 basic_session_run_hooks.py:606] Saving checkpoints for 0 into ../model/gru_seq2seq/model.ckpt.


Reading ../data/train.tsv


I0815 01:03:34.743826 140072451344256 basic_session_run_hooks.py:262] loss = 10.9506855, step = 0
I0815 01:03:34.745883 140072451344256 basic_session_run_hooks.py:262] lr = 0.0004
I0815 01:04:25.743890 140072451344256 basic_session_run_hooks.py:692] global_step/sec: 1.96076
I0815 01:04:25.749890 140072451344256 basic_session_run_hooks.py:260] loss = 2.2924652, step = 100 (51.006 sec)
I0815 01:04:25.760263 140072451344256 basic_session_run_hooks.py:260] lr = 0.00039837044 (51.014 sec)
I0815 01:05:13.621898 140072451344256 basic_session_run_hooks.py:692] global_step/sec: 2.08864
I0815 01:05:13.626769 140072451344256 basic_session_run_hooks.py:260] loss = 1.2462063, step = 200 (47.877 sec)
I0815 01:05:13.631274 140072451344256 basic_session_run_hooks.py:260] lr = 0.00039674752 (47.871 sec)
I0815 01:06:03.284174 140072451344256 basic_session_run_hooks.py:692] global_step/sec: 2.01361
I0815 01:06:03.287893 140072451344256 basic_session_run_hooks.py:260] loss = 0.83549994, step = 300 (49.661

...

Reading ../data/test.tsv


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
I0815 09:47:45.404990 140072451344256 interactiveshell.py:2882] Evaluation: Testing (Exact Match) Accuracy: 0.742
I0815 09:47:45.406592 140072451344256 interactiveshell.py:2882] Best (Exact Match) Accuracy: 0.742
I0815 09:47:45.513877 140072451344256 estimator.py:1145] Calling model_fn.
I0815 09:47:45.912744 140072451344256 saver.py:1499] Saver not created because there are no variables in the graph to restore
I0815 09:47:46.881506 140072451344256 saver.py:1499] Saver not created because there are no variables in the graph to restore


[<tf.Variable 'Embedding/dense/kernel:0' shape=(1024, 300) dtype=float32_ref>,
 <tf.Variable 'Embedding/dense/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'Embedding/glove:0' shape=(8692, 300) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional/forward_cu_dnngru/kernel:0' shape=(600, 900) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/forward_cu_dnngru/recurrent_kernel:0' shape=(300, 900) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/forward_cu_dnngru/bias:0' shape=(1800,) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/backward_cu_dnngru/kernel:0' shape=(600, 900) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/backward_cu_dnngru/recurrent_kernel:0' shape=(300, 900) dtype=float32>,
 <tf.Variable 'Encoder/bidirectional/backward_cu_dnngru/bias:0' shape=(1800,) dtype=float32>,
 <tf.Variable 'Decoder/memory_layer/kernel:0' shape=(600, 300) dtype=float32_ref>,
 <tf.Variable 'Decoder/memory_layer_1/kernel:0' shape=(600, 300) dtype=float32_ref>,
 <tf.Var

I0815 09:47:50.405172 140072451344256 estimator.py:1147] Done calling model_fn.
I0815 09:47:50.409385 140072451344256 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0815 09:47:50.838192 140072451344256 monitored_session.py:240] Graph was finalized.
I0815 09:47:50.857290 140072451344256 saver.py:1280] Restoring parameters from ../model/gru_seq2seq/model.ckpt-44010
I0815 09:47:52.232073 140072451344256 session_manager.py:500] Running local_init_op.
I0815 09:47:52.360752 140072451344256 session_manager.py:502] Done running local_init_op.
I0815 09:47:59.019027 140072451344256 basic_session_run_hooks.py:606] Saving checkpoints for 44010 into ../model/gru_seq2seq/model.ckpt.


KeyboardInterrupt: ignored